In [1]:
import numpy as np
import os
import sqlite3
import gdstk
from tqdm import tqdm

base_dir = os.getcwd()
DB_PATH = os.path.join(base_dir, "data", "Main.db")
uri_path = f"file:{DB_PATH}?mode=ro"
conn = sqlite3.connect(uri_path, uri=True)
cursor = conn.cursor()

ids = np.load("id.npy", allow_pickle=True)

id_list = np.unique(ids)

lib = gdstk.Library(unit=1e-6, precision=1e-9)
top = lib.new_cell("TOP")

r = 1.8e-3
single = 0.4e-6
U = int(r*2/single)

x = np.linspace(-(r-0.5*single), (r-0.5*single), U)
y = np.linspace(-(r-0.5*single), (r-0.5*single), U)
X, Y = np.meshgrid(x, y)

ids[np.sqrt(X**2 + Y**2)>r] = np.nan

In [2]:
def Create_template(id, cursor, lib):
    cursor.execute('SELECT class, parameterA, parameterB, parameterC FROM Parameter WHERE ID=(?)', (id,))
    row = cursor.fetchone()
    Class, parameterA, parameterB, parameterC = row
    if Class == 1:
        radius = parameterA*1e6
        cell_name = f"CIRCLE_{radius:.6f}"
        cell = lib.new_cell(cell_name)
        circle = gdstk.ellipse((0, 0), radius, tolerance=1e-3)
        cell.add(circle)
    elif Class == 2:
        long = parameterA*1e6
        cell_name = f"RECT_{long:.6f}"
        cell = lib.new_cell(cell_name)
        half = long/2
        rect = gdstk.rectangle((-half, -half),( half,  half))
        cell.add(rect)
    elif Class == 3:
        long, short = parameterA*1e6, parameterB*1e6
        cell_name = f"Cross_{long:.6f}_{short:.6f}"
        cell = lib.new_cell(cell_name)
        half_long = long/2
        half_short = short/2
        rect_h = gdstk.rectangle((-half_long, -half_short),( half_long,  half_short))
        rect_v = gdstk.rectangle((-half_short, -half_long),( half_short,  half_long))
        cell.add(rect_h)
        cell.add(rect_v)
    elif Class == 4:
        long, short, radius = parameterA*1e6, parameterB*1e6, parameterC
        cell_name = f"Fishnet_{long:.6f}_{short:.6f}_{radius:.6f}"
        cell = lib.new_cell(cell_name)
        half_long = long/2
        half_short = short/2
        rect_h = gdstk.rectangle((-half_long, -half_short),( half_long,  half_short))
        rect_v = gdstk.rectangle((-half_short, -half_long),( half_short,  half_long))
        circle = gdstk.ellipse((0, 0), radius, tolerance=1e-3)
        cell.add(rect_h)
        cell.add(rect_v)
        cell.add(circle)
    return cell

In [3]:
for id in tqdm(id_list):
    cell = Create_template(id, cursor, lib)
    positions = np.argwhere(ids == id)
    for po in positions:
        x, y = po
        ref = gdstk.Reference(cell, origin=(x, y))
        top.add(ref)

lib.write_gds("physical_parameter_circles.gds")
print("GDS 文件已生成")

100%|██████████| 823/823 [02:07<00:00,  6.47it/s]


GDS 文件已生成
